<a href="https://colab.research.google.com/github/kvamsi7/gen-ai-apps/blob/main/YouTube_reseach_reports_in_GMail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet composio-gemini composio_llamaindex llama-index-llms-gemini

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.8/477.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.6/250.6 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.9/164.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 52.3 MB/s eta 0:00:00
   ━━━━━━

In [10]:
from google.genai import types
from google import genai
from composio_gemini import Action, ComposioToolSet, App
from google.colab import userdata

client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))
toolset = ComposioToolSet(api_key=userdata.get('COMPOSIO_API_KEY'))

In [ ]:
!composio login

In [ ]:
!composio add youtube

In [ ]:
!composio add gmail

In [12]:
from composio_llamaindex import ComposioToolSet, App, Action
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.llms import ChatMessage
from llama_index.llms.gemini import Gemini

# Initialize toolset and LLM
toolset = ComposioToolSet(api_key=userdata.get('COMPOSIO_API_KEY'))
tools = toolset.get_tools(actions=[Action.YOUTUBE_SEARCH_YOU_TUBE,
                                   Action.GMAIL_CREATE_EMAIL_DRAFT,
                                   Action.YOUTUBE_VIDEO_DETAILS])

function_calling_llm = Gemini(model="models/gemini-2.0-flash-exp", api_key=userdata.get('GOOGLE_API_KEY'))

# Setup chatbot-style prefix messages
def create_prefix_message():
    return [
        ChatMessage(
            role="system",
            content=(
                """
                You are an expert at searching through YouTube videos,
                retrieving video details, and summarizing those details
                into a report that you send as an email.
                """
            ),
        ),
    ]

prefix_messages = create_prefix_message()

# Initialize the agent
agent = FunctionCallingAgentWorker(
    tools=tools, # type: ignore
    llm=function_calling_llm,
    prefix_messages=prefix_messages,
    max_function_calls=20,
    allow_parallel_tool_calls=True,
    verbose=True,
).as_agent()

# Task-specific logic in a chatbot-like flow
def chatbot():
    print("🤖: Hi! I can help you research content based on the latest trends. Let’s start!")
    human_input = input("What do you want to research: ")
    questions = function_calling_llm.complete("Create search terms for the topic: " + human_input)
    res = agent.chat(str(questions)+"\n These are the YouTube search terms for the topic you need to research about, they are related to the topic: " + human_input+". After your YouTube video research is complete, please make sure to create a detailed summary of all of the research. Make sure to list all of the URLs and titles for all of the videos you referenced as citations at the bottom of your email, and save summary as a draft email addressed to webpaige@google.com.")
    #print(res.response)

if __name__ == "__main__":
    chatbot()

🤖: Hi! I can help you research content based on the latest trends. Let’s start!
What do you want to research: Show me best reinforcement tutorials
Added user message to memory: Okay, here are some search terms for finding the best reinforcement learning tutorials, categorized for clarity:

**General & Broad:**

*   Reinforcement learning tutorial
*   Best reinforcement learning tutorials
*   Learn reinforcement learning
*   Reinforcement learning for beginners
*   Introduction to reinforcement learning

**Specificity & Level:**

*   Reinforcement learning tutorial beginner
*   Reinforcement learning tutorial intermediate
*   Reinforcement learning tutorial advanced
*   Reinforcement learning tutorial step-by-step
*   Reinforcement learning tutorial with examples
*   Reinforcement learning tutorial hands-on

**Focus on Specific Algorithms/Techniques:**

*   Q-learning tutorial
*   Deep Q-Network (DQN) tutorial
*   Policy gradient tutorial
*   Actor-Critic tutorial
*   SARSA tutorial
*  

In [13]:
import jax
import jax.numpy as jnp
from jax import random

# Check for GPU availability.  This is crucial.
if jax.default_backend() != 'gpu':
    print("WARNING: No GPU detected. Code will run on CPU, which will be much slower.")
    # Optionally, raise an exception to halt execution if GPU is required.
    # raise RuntimeError("GPU is required for this example.")

# Generate a random key.  This is how JAX handles randomness.
key = random.PRNGKey(0)

# Create two large random matrices on the GPU (if available).
key, subkey1, subkey2 = random.split(key, 3)  # Split the key for multiple uses.
matrix_a = random.normal(subkey1, (20000, 20000))
matrix_b = random.normal(subkey2, (20000, 20000))

# Perform matrix multiplication on the GPU.
matrix_c = jnp.dot(matrix_a, matrix_b)

# Force the computation to complete and copy the result back to the CPU.
# Without block_until_ready, JAX's asynchronous execution might not show the full
# GPU utilization immediately.
matrix_c = matrix_c.block_until_ready()

# Print the first element of the result (just to show something).
print("First element of the result:", matrix_c[0, 0])


# A more concise version, using jax.jit for potential speedup
@jax.jit
def matrix_multiply(key):
    subkey1, subkey2 = random.split(key)
    matrix_a = random.normal(subkey1, (10000, 10000))
    matrix_b = random.normal(subkey2, (10000, 10000))
    return jnp.dot(matrix_a, matrix_b)

key = random.PRNGKey(42)
result_jit = matrix_multiply(key)
result_jit.block_until_ready() # Important:  Ensure the JIT-compiled function is ready.
print("First element of JIT-compiled result:", result_jit[0, 0])

First element of the result: 40.193592
First element of JIT-compiled result: -43.019566
